In [1]:
import os
import pandas as pd

In [5]:
path_bank_clients = os.getcwd() + '/1.0/volumes/clickhouse/samples/bank_clients.tsv'
path_bank_complaints = os.getcwd() + '/1.0/volumes/clickhouse/samples/bank_complaints.tsv'
path_bank_transactions = os.getcwd() + '/1.0/volumes/clickhouse/samples/bank_transactions.tsv'
path_ecosystem_mapping = os.getcwd() + '/1.0/volumes/clickhouse/samples/ecosystem_mapping.tsv'
path_market_place_delivery = os.getcwd() + '/1.0/volumes/clickhouse/samples/market_place_delivery.tsv'
path_mobile_build = os.getcwd() + '/1.0/volumes/clickhouse/samples/mobile_build.tsv'
path_mobile_clients = os.getcwd() + '/1.0/volumes/clickhouse/samples/mobile_clients.tsv'

In [8]:
os.getcwd()
path_bank_clients

'/Users/olgakamskaa/Sber_school/1.0/volumes/clickhouse/samples/bank_clients.tsv'

In [6]:
df_bank_clients = pd.read_csv(path_bank_clients, sep='\t')
df_bank_complaints = pd.read_csv(path_bank_complaints, sep='\t')
df_bank_transactions = pd.read_csv(path_bank_transactions, sep='\t')
df_ecosystem_mapping = pd.read_csv(path_ecosystem_mapping, sep='\t')
df_market_place_delivery = pd.read_csv(path_market_place_delivery, sep='\t')
df_mobile_build = pd.read_csv(path_mobile_build, sep='\t')
df_mobile_clients = pd.read_csv(path_mobile_clients, sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/olgakamskaa/Sber_school/1.0/volumes/clickhouse/samples/bank_clients.tsv'

In [4]:
database_name = 'database_name'
table_name = 'table_name'

In [5]:
def to_sql_many(data, database_name, table_name, row_batch=10000):

    sql = '''insert into {}.{} ({}) values {}'''

    if len(data.index) == 0:
        return 'empty file'

    table_cols = ', '.join(col for col in list(data.columns))

    data_len = len(data)
    for part in range((data_len // row_batch) + 1):
        print(part, '/', data_len // row_batch)

        inserts = list([list(i) for i in data[part * row_batch:(part + 1) * row_batch].values])
        query = sql.format(database_name, table_name, table_cols, inserts)
        query = query.replace('[[', '(')\
        .replace(']]', ')')\
        .replace('[', '(')\
        .replace(']', ')')\
        .replace('nan', 'NULL')

    return query

# to_sql_many(df_bank_clients, database_name, table_name, row_batch=10000)

In [81]:
# Клиент который обратился с проблемой
df_bank_complaints.merge(df_bank_clients, left_on='uerId', right_on='userId', how='left')

,event_date,uerId,text,userId,accout,phone,fio
0,2020-05-21 12:34:45,b2857395394,У меня пропали с карты 15000р.,b2857395394,40101810045250010041,79291234567,Петров А.Б.


In [56]:
# Клиент который обращался с проблемой
df_mobile_build[
    (df_mobile_build['event_date'] >= '2020-05-21 12:00:00')
    & (df_mobile_build['event_date'] < '2020-05-21 13:00:00')
]

,event_date,from_call,to_call,duration_sec
0,2020-05-21 12:04:11,79297654321,79291234567,362


In [62]:
# Данные клиента в таблице, вытаскиваем сведения по client_id
# Видим что клиент общался с сотрудником Петров А.Б.
df_mobile_clients[
    df_mobile_clients['phone'].isin(['79297654321', '79291234567'])
]

,client_id,phone,fio,address
0,m1395827402,79291234567,Петров А.Б.,"г. Москва, Улица Ленина, д. 1 кв. 23"
1,m2284039583,79297654321,Иванов В.Г.,\N


In [63]:
# По данным mobile_user_id получаем данные по market_plece_user_id
df_ecosystem_mapping[
    df_ecosystem_mapping['mobile_user_id'].isin(['m2284039583', 'm1395827402'])
]

,id,bank_id,mobile_user_id,market_plece_user_id
0,18,\N,m2284039583,mp2920493810
1,49,b2857395394,m1395827402,mp1850634567


In [82]:
# Вытаскиваем номер телефона? контактные данные. По ФИО и юзер id в мобильном приложении
df_market_place_delivery[
    (df_market_place_delivery['user_id'].isin(['mp2920493810','mp1850634567']))
    & (df_market_place_delivery['contact_fio'].isin(['Иванов В.Г.']))
]

,event_date,user_id,contact_fio,contact_phone,address
0,2020-05-07 15:21:45,mp2920493810,Иванов В.Г.,79297483321,"г. Москва, ул. Маршала Жукова, д. 7, офис 12"
1,2020-05-12 16:01:23,mp2920493810,Иванов В.Г.,79297483321,"г. Москва, ул. Маршала Жукова, д. 7, офис 12"


In [84]:
df_market_place_delivery[
    (df_market_place_delivery['address'].isin(['г. Москва, ул. Маршала Жукова, д. 7, офис 12']))
]

,event_date,user_id,contact_fio,contact_phone,address
0,2020-05-07 15:21:45,mp2920493810,Иванов В.Г.,79297483321,"г. Москва, ул. Маршала Жукова, д. 7, офис 12"
1,2020-05-12 16:01:23,mp2920493810,Иванов В.Г.,79297483321,"г. Москва, ул. Маршала Жукова, д. 7, офис 12"


In [91]:
# Одновамильцы вора
df_bank_clients[
    (df_bank_clients['fio'].apply(lambda x: x.find('Иванов') >= 0))
    | (df_bank_clients['phone'].isin(['79297483321']))
]

,userId,accout,phone,fio
5,b3432616287,40101810045251519357,79035324827,Иванова Ж. М.
21,b3296469953,40101810045254833851,79161271688,Иванов О. Т.
24,b8529433882,40101810045253981885,79262613635,Иванова Б. З.
31,b4583452231,40101810045253303429,79263030534,Иванова Д. Д.
98,b5295134029,40101810045259536980,79036052831,Иванов К. И.
133,b2247990215,40101810045256373585,79031347551,Иванова А. Е.
